In [1]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict


In [2]:
DNS_lookups = {
    'nest_therm': '.nest.com',
    'nest_cam': '.dropcam.com',
    'smartthings': '.smartthings.com',
    'pixstar': '.pix-star.com',
    'echo': 'device-metrics-us.amazon.com',
    'sharx': 'sharxsecurity.com',
    'august': '.august.com',
    'hue': '.meethue.com',
    #'apple': 'time-ios.apple.com',
    'apple': 'gs-loc.ls-apple.com',
    'ps': '.playstation.net',
    'xbox': 'xboxlive.com',
    'nintendo': 'nintendo.net',
}

DNS_strict_lookups = {
    'nest_therm': 'devices.nest.com',
    'nest_cam': 'nexus.dropcam.com',
    'smartthings': 'api.smartthings.com',
    'pixstar': 'api.pix-star.com',
    'echo': 'device-metrics-us.amazon.com',
    'sharx': 'sharxsecurity.com',
    'august': 'doorbells.august.com',
    'hue': 'bridge.meethue.com',
    'apple': 'gs-loc.ls-apple.com',
    'ps': 'update.playstation.net',
    'xbox': '.xboxlive.com',
    'nintendo': '.nintendo.net',
}

# SRCIP set 1 from 1 min full pcap

In [3]:
df = pd.read_csv("pima-extract.csv", sep="|").fillna('')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
ip_list = []
for domain in DNS_lookups.values():
    ip_list.extend( list( df [ df['dns.qry.name'].str.contains(domain) ]['ip.src'].unique()) )

In [5]:
ip_set1 = set( ip_list )

In [6]:
len(ip_set1)

6448

# SRCIP set 2 from 30 min filtered pcap

In [7]:
df = pd.read_csv("pima-201606291900.csv", sep=",", names=['ip.src', 'ipv6.src', 'type', 'domain']).fillna('')

df['domain'] = df['domain'].str.lower()

In [8]:
df.head()

,ip.src,ipv6.src,type,domain
0.001390,69.137.179.22,,0x0001,us-prof.np.community.playstation.net
0.006040,,2601:500:8500:9e20:a915:7cb1:7eea:6f99,0x0001,device-metrics-us.amazon.com
0.059405,69.242.233.136,,0x0001,device-metrics-us.amazon.com
0.061309,71.228.156.2,,0x0001,userpresence.xboxlive.com
0.083694,107.2.19.87,,0x0001,transport04-rts16-iad01.transport.home.nest.com


In [9]:
ip_list = ['69.244.36.23',
 '69.244.45.171',
 '69.244.8.159',
 '73.24.155.23',
 '73.24.169.163',
 '98.225.66.206']

df[df['ip.src']==ip_list[5]]

,ip.src,ipv6.src,type,domain
1203.079644,98.225.66.206,,0x0001,www.xboxlive.com
1203.093698,98.225.66.206,,0x0001,eplists.xboxlive.com


In [10]:
ip_list = list(df['ip.src'].unique())

In [11]:
ip_set2 = set(ip_list)

In [12]:
len(ip_set2)

11736

# PRIORITY LIST

In [13]:
df_IP = pd.DataFrame({'ip.src': df['ip.src'].unique()}).sort_values(by='ip.src').set_index('ip.src')

In [14]:
ip_dns = defaultdict(int)
for domain in DNS_lookups.values():
    #ip_dns[domain] = df['dns.qry.name'].str.contains(domain)
    temp = pd.DataFrame( { 'ip.src' : df[ df['domain'].str.contains(domain) ]['ip.src'].unique(),
              domain: True} ).set_index('ip.src')
    df_IP[domain]=temp
    df_IP[domain].fillna(False, inplace=True)

In [15]:
df_IP.head()

,.meethue.com,gs-loc.ls-apple.com,device-metrics-us.amazon.com,sharxsecurity.com,nintendo.net,.dropcam.com,.august.com,.playstation.net,.smartthings.com,.pix-star.com,xboxlive.com,.nest.com
ip.src,,,,,,,,,,,,
,True,True,True,False,True,True,False,True,True,True,True,True
107.2.0.102,False,False,True,False,False,False,False,False,False,False,False,False
107.2.0.110,False,False,True,False,False,False,False,False,False,False,True,False
107.2.0.117,False,False,False,False,False,False,False,False,False,False,True,False
107.2.0.127,False,False,False,False,False,False,False,False,False,False,True,False


In [52]:
# PIMA TOP 18
ip_list = []

for ippcap in os.listdir("/Users/sgrove201/Projects/dpi-pcap/pima-top20_24hr_Aug18/processed_00000002/"):
    ip_list.append(ippcap.rstrip('.pcap'))
    
                   
print ip_list
df_filt = df_IP.ix[ip_list]

writer = pd.ExcelWriter('PIMA_ip_vs_domain.xlsx', engine='xlsxwriter')
df_filt.to_excel(writer, 'Sheet1')
writer.save()

df_filt.sum()

['107.2.26.135', '107.2.7.171', '24.11.146.86', '24.11.147.79', '24.23.77.2', '69.136.124.166', '69.136.125.106', '69.137.181.46', '69.244.12.6', '69.244.34.131', '69.244.59.102', '73.24.154.94', '73.24.161.206', '73.24.165.99', '73.24.251.90', '73.24.253.138', '98.225.111.78', '98.225.94.23']


.meethue.com                     2
gs-loc.ls-apple.com              1
device-metrics-us.amazon.com    14
sharxsecurity.com                0
nintendo.net                     2
.dropcam.com                     0
.august.com                      0
.playstation.net                 9
.smartthings.com                 1
.pix-star.com                    0
xboxlive.com                     6
.nest.com                        7
dtype: int64

In [16]:
df_IP.sum()

.meethue.com                      98
gs-loc.ls-apple.com             3486
device-metrics-us.amazon.com    3870
sharxsecurity.com                  0
nintendo.net                     759
.dropcam.com                      38
.august.com                        7
.playstation.net                1689
.smartthings.com                  65
.pix-star.com                     45
xboxlive.com                    2489
.nest.com                        973
dtype: int64

In [17]:
indx = df_IP.sum(axis=1).sort_values(ascending=False).index

In [18]:
df_IP_sorted = df_IP.reindex(indx)
df_IP_sorted['total'] = df_IP_sorted.sum(axis=1)

In [19]:
df_IP_sorted.to_pickle("pima-201606291900-sorted-IoT-IPs.pkl")

In [20]:
print "IPs with IoTs:", len(df_IP_sorted[ df_IP_sorted.total>0 ])
print "IPs without IoTs:", len(df_IP_sorted[ df_IP_sorted.total==0 ])

IPs with IoTs: 11736
IPs without IoTs: 0


In [21]:
# Distribution of number of IoTs per home
tot = df_IP_sorted[ df_IP_sorted.total>0 ]['total']

In [22]:
from collections import Counter
print "Distribution of number of IoTs per home: ", Counter(tot)

Distribution of number of IoTs per home:  Counter({1: 10215, 2: 1337, 3: 156, 4: 16, 10: 5, 5: 4, 11: 2, 6: 1})


In [23]:
interesting = df_IP_sorted[df_IP_sorted.total > 2]
print len(interesting)
interesting.sum().sort_values(ascending=False)

184


total                           630.0
device-metrics-us.amazon.com    130.0
xboxlive.com                    119.0
gs-loc.ls-apple.com             100.0
.playstation.net                 72.0
.nest.com                        71.0
nintendo.net                     55.0
.meethue.com                     37.0
.smartthings.com                 23.0
.dropcam.com                     12.0
.pix-star.com                     8.0
.august.com                       3.0
sharxsecurity.com                 0.0
dtype: float64

# UNION of ip lists in PIMA

In [24]:
ip_set = ip_set1 | ip_set2

In [25]:
len(ip_set)

17577

In [26]:
with open('pima-iot-cpeip.csv', 'w') as fout:
    for ip in ip_set:
        fout.write(ip)
        fout.write('\n')

In [27]:
len( ip_set1 & ip_set2 )

607

# UNION of top 100 ip lists in PIMA

In [28]:
df_temp = pd.read_pickle("pima-extract-sorted-IoT-IPs.pkl")

top_100_1 = list(df_temp.iloc[:100].index)

In [29]:
top_100_2 = list(df_IP_sorted.iloc[:100].index)

In [30]:
set(top_100_1) & set(top_100_2)

{'',
 '69.241.95.82',
 '69.241.95.83',
 '69.241.95.84',
 '69.241.95.85',
 '69.241.95.86',
 '69.241.95.87'}

In [31]:
df1 = df_temp[df_temp['total']>0]['total'].reset_index()
df2 = df_IP_sorted[df_IP_sorted['total']>0]['total'].reset_index()

In [32]:
# merge by taking max total
df = df1.merge(df2, on='ip.src')
df['total'] = df[['total_x', 'total_y']].max(axis=1)

# remove NULL IP and sort
df = df[df['ip.src']!='']
df = df.sort_values(by='total',ascending=False).set_index('ip.src')

In [33]:
top_100_ips = list( df.iloc[:100].index )

In [34]:
#f = open('pima-top_100_IoT-IPs.csv', 'w')
#
#for ip in top_100_ips:
#    f.write(ip+'\n')
#    print ip
#    
#f.close()

In [35]:
list( df.iloc[:100].index )

['69.241.95.82',
 '69.241.95.84',
 '69.241.95.83',
 '69.241.95.85',
 '69.241.95.86',
 '69.241.95.87',
 '98.225.83.234',
 '24.23.67.116',
 '69.137.181.46',
 '69.242.228.100',
 '107.2.18.120',
 '69.244.46.39',
 '73.24.234.78',
 '73.24.165.99',
 '76.18.106.4',
 '73.24.154.165',
 '71.226.42.67',
 '107.2.8.57',
 '71.226.47.97',
 '73.24.249.230',
 '73.24.188.250',
 '69.244.41.193',
 '69.244.15.55',
 '24.23.77.2',
 '73.24.242.243',
 '96.91.74.54',
 '107.2.27.117',
 '76.18.96.253',
 '68.84.193.5',
 '69.244.35.157',
 '71.226.125.95',
 '69.244.59.102',
 '76.18.106.58',
 '73.24.254.175',
 '69.244.38.245',
 '24.11.146.155',
 '73.24.253.138',
 '76.18.106.67',
 '24.11.146.86',
 '98.225.122.116',
 '76.18.106.42',
 '76.18.106.53',
 '73.24.250.68',
 '71.228.145.4',
 '73.24.234.152',
 '69.244.34.131',
 '69.244.23.21',
 '69.244.32.163',
 '107.2.13.162',
 '69.244.8.100',
 '24.11.147.79',
 '73.24.251.90',
 '69.252.212.86',
 '98.225.68.127',
 '73.24.215.23',
 '69.136.124.166',
 '107.2.20.173',
 '73.24.214.1